In [17]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import openweathermapy.core as owm
#import os
#import csv
import seaborn as sns

import gmaps
import gmaps.geojson_geometries

from Project_Gas_Library import f_EIA_ListSeries, f_EIA_ExtractData, f_PlotData, f_EIA_PlotData, \
f_WorldWeather_ExtractData, f_PlotData, f_Census_ExtractData, calculate_color

from datetime import datetime
from dateutil.relativedelta import relativedelta
from calendar import monthrange

from config import worldweather_key, g_key, EIA_key, census_key

from matplotlib.pyplot import cm
from citipy import citipy
from pprint import pprint

from matplotlib.cm import viridis
from matplotlib.colors import to_hex

from census import Census
import os

In [18]:
url = "http://api.eia.gov/category/"
category_id = "714805" #Natural Gas
eia_api_key = EIA_key

list_ser = f_EIA_ListSeries(url,eia_api_key,category_id)
df = pd.DataFrame(list_ser)
df.to_csv("../../Data/CSV_Files/EIA_Series_ID.csv", encoding='utf-8')
df

,f,name,series_id,units,updated
0,A,U.S. Total Natural Gas Injections into Undergr...,NG.N5050US2.A,Million Cubic Feet,30-APR-19 01.42.23 PM
1,M,U.S. Total Natural Gas Injections into Undergr...,NG.N5050US2.M,Million Cubic Feet,28-JUN-19 12.40.59 PM
2,A,U.S. Natural Gas Underground Storage Withdraw...,NG.N5060US2.A,Million Cubic Feet,30-APR-19 01.42.23 PM
3,M,U.S. Natural Gas Underground Storage Withdraw...,NG.N5060US2.M,Million Cubic Feet,28-JUN-19 12.40.59 PM
4,A,"U.S. Dry Natural Gas Production, Annual",NG.N9070US2.A,Million Cubic Feet,28-JUN-19 12.40.59 PM
5,M,"U.S. Dry Natural Gas Production, Monthly",NG.N9070US2.M,Million Cubic Feet,28-JUN-19 12.40.59 PM
6,A,"U.S. Natural Gas Input Supplemental Fuels, Annual",NG.N9090US2.A,Million Cubic Feet,28-JUN-19 12.40.59 PM
7,M,"U.S. Natural Gas Input Supplemental Fuels, Mon...",NG.N9090US2.M,Million Cubic Feet,28-JUN-19 12.40.59 PM
8,A,"U.S. Natural Gas Total Consumption, Annual",NG.N9140US2.A,Million Cubic Feet,28-JUN-19 12.40.59 PM
9,M,"U.S. Natural Gas Total Consumption, Monthly",NG.N9140US2.M,Million Cubic Feet,28-JUN-19 12.40.59 PM


In [19]:
# df_ng_consump = df.loc[(df["name"].str.contains("Consumption|Deliveries to Electric Power Consumers")==True) & (df["f"]=="A")]
# df_ng_consump = df_ng_consump.drop_duplicates(subset="series_id",keep="first")
# df_ng_consump = df_ng_consump.drop_duplicates(subset="name",keep="first")
# df_ng_consump

In [20]:
df_ng_consump = df.loc[(df["name"].str.contains("Total Consumption")==True) & (df["f"]=="A")]
f_ng_consump = df_ng_consump.drop_duplicates(subset="series_id",keep="first")
df_ng_consump = df_ng_consump.drop_duplicates(subset="name",keep="first")
df_ng_consump

,f,name,series_id,units,updated
8,A,"U.S. Natural Gas Total Consumption, Annual",NG.N9140US2.A,Million Cubic Feet,28-JUN-19 12.40.59 PM
26,A,"Alabama Natural Gas Total Consumption, Annual",NG.NA1490_SAL_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
34,A,"Alaska Natural Gas Total Consumption, Annual",NG.NA1490_SAK_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
47,A,"Arizona Natural Gas Total Consumption, Annual",NG.NA1490_SAZ_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
59,A,"Arkansas Natural Gas Total Consumption, Annual",NG.NA1490_SAR_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
70,A,"California Natural Gas Total Consumption, Annual",NG.NA1490_SCA_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
82,A,"Colorado Natural Gas Total Consumption, Annual",NG.NA1490_SCO_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
89,A,"Connecticut Natural Gas Total Consumption, Annual",NG.NA1490_SCT_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
97,A,"Delaware Natural Gas Total Consumption, Annual",NG.NA1490_SDE_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM
101,A,District of Columbia Natural Gas Total Consump...,NG.NA1490_SDC_2.A,Million Cubic Feet,28-SEP-18 01.18.20 PM


In [21]:
eia_api_key = EIA_key
url = "http://api.eia.gov/series/"

flag = True

for ind, val in df_ng_consump.iterrows():
    series_id = val["series_id"]    
    valx, valy, title, country, unit = f_EIA_ExtractData(url,eia_api_key,series_id)
    valy = [x/1000 for x in valy]
    
    finpos = title.find("Natural") - 1
    data = pd.DataFrame({'Date':valx,title[0:finpos]:valy})
    data = data.set_index("Date")
    if flag:
        df = data
        flag = False
    else:
        df = df.merge(data, how='outer', left_index=True, right_index=True)

df.dropna(how='any', inplace=True)
df

,U.S.,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Date,,,,,,,,,,,,,,,,,,,,,
1999-06-01,22405.151,337.270,422.816,165.076,252.853,2339.521,333.085,152.237,56.013,32.281,...,35.794,278.841,4009.689,159.889,8.033,276.793,287.302,139.961,380.560,96.726
2000-06-01,23333.121,353.614,427.288,205.235,251.329,2508.797,367.920,159.712,48.387,33.468,...,37.939,270.658,4421.777,164.557,10.426,268.770,286.653,147.854,393.601,101.314
2001-06-01,22238.624,332.693,408.960,240.812,227.943,2464.565,463.738,146.278,50.113,29.802,...,37.077,255.990,4252.152,159.299,7.919,237.853,312.114,141.090,359.784,98.569
2002-06-01,23027.021,379.343,419.131,250.734,242.325,2273.193,459.397,177.587,52.216,32.898,...,41.577,255.515,4303.831,163.379,8.367,258.202,233.716,146.455,385.310,112.872
2003-06-01,22276.502,350.345,414.234,272.921,246.916,2269.405,436.253,154.075,46.177,32.814,...,43.881,257.315,4050.632,154.125,8.400,262.970,249.599,126.986,394.711,115.358
2004-06-01,22402.546,382.367,406.319,349.622,215.124,2406.889,440.378,162.642,48.057,32.227,...,41.679,231.133,3908.243,155.891,8.685,277.434,262.485,122.267,383.316,107.060
2005-06-01,22014.434,353.156,432.972,321.584,213.609,2248.256,470.321,168.067,46.904,32.085,...,42.555,230.338,3503.636,160.275,8.372,299.746,264.754,117.136,410.250,108.314
2006-06-01,21699.071,391.093,373.850,358.069,233.868,2315.721,450.832,172.682,43.190,29.049,...,40.739,221.626,3432.236,187.399,8.056,274.175,263.395,113.084,372.462,108.481
2007-06-01,23103.793,418.512,369.967,392.954,226.439,2395.674,504.775,180.181,48.155,32.966,...,53.938,221.118,3516.706,219.700,8.867,319.913,272.613,115.974,398.370,140.912


In [31]:
# path = "Map"
file = "../../Data/Map/us-states.geojson"

with open(file) as f:
    data = json.load(f)
    
for feature in data['features']:
#     print(feature['geometry']['type'])
#     print(feature['geometry']['coordinates'])
    state = feature['properties']['name']
    if state in list(df.columns):
        feature['properties']['NatGasComp'] = [float(x) for x in list(df[state].values)]
    else:
        feature['properties']['NatGasComp'] = [0.0] * (2017-1999+1)
for feature in data['features']:
    print(feature['properties']['name'], \
        feature['properties']['NatGasComp'])

data_mod = json.dumps(data)

with open("../../Data/Map/us-states.js",'w') as f:
    f.write(data_mod)
    
with open("../../static/js/us-states.js",'w') as f:
    f.write(data_mod)

Alabama [337.27, 353.614, 332.693, 379.343, 350.345, 382.367, 353.156, 391.093, 418.512, 404.157, 454.456, 534.779, 598.514, 666.712, 615.407, 635.323, 681.149, 694.881, 669.088]
Alaska [422.816, 427.288, 408.96, 419.131, 414.234, 406.319, 432.972, 373.85, 369.967, 341.888, 342.261, 333.312, 335.458, 343.11, 332.298, 328.945, 333.602, 330.552, 345.941]
Arizona [165.076, 205.235, 240.812, 250.734, 272.921, 349.622, 321.584, 358.069, 392.954, 399.188, 369.739, 330.914, 288.802, 332.068, 332.073, 306.715, 351.263, 360.576, 321.638]
Arkansas [252.853, 251.329, 227.943, 242.325, 246.916, 215.124, 213.609, 233.868, 226.439, 234.901, 244.193, 271.515, 284.076, 296.132, 282.12, 268.444, 291.006, 309.732, 317.684]
California [2339.521, 2508.797, 2464.565, 2273.193, 2269.405, 2406.889, 2248.256, 2315.721, 2395.674, 2405.266, 2328.504, 2273.128, 2153.186, 2403.494, 2415.571, 2339.392, 2301.217, 2172.889, 2110.829]
Colorado [333.085, 367.92, 463.738, 459.397, 436.253, 440.378, 470.321, 450.832, 50